In [2]:
import pandas as pd

# Load data from the CSV file
df = pd.read_csv('c:/Users/akhil/OneDrive/Desktop/result.csv')  # Replace 'result.csv' with your actual file name

# Replace "section_id" with the actual column name you have

# Extract the starting word from the "section_id" column
df['section_id_start'] = df['Section_id'].str.split().str[0]

# Columns to compare with "section_id_start"
columns_to_compare = ['PRIMARY_INTERVENTION', 'PRIMARY_ELIGIBILITY', 'PRIMARY_RESULTS',
                      'PRIMARY_ADVERSE_EVENTS', 'SECONDARY_INTERVENTION', 'SECONDARY_ELIGIBILITY',
                      'SECONDARY_RESULTS', 'SECONDARY_ADVERSE_EVENTS']

# Loop through the columns and check if the starting word matches
for column in columns_to_compare:
    mask = df['section_id_start'] != df[column].str.split().str[0]
    df.loc[mask, column] = None

# Remove the temporary column used for comparison
df.drop('section_id_start', axis=1, inplace=True)

df.to_csv('c:/Users/akhil/OneDrive/Desktop/updated_result.csv', index=False)


In [10]:
#insert all the combined data in combine_data table
import csv
import mysql.connector

# Establish a MySQL connection
connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database = 'clinical_data'
)

connection.database = 'clinical_data'
csv_file_path = "C:/Users/akhil/OneDrive/Desktop/updated_result.csv"

with open(csv_file_path, 'r', encoding='utf-8') as file:
    csv_data = csv.reader(file)
    header = next(csv_data)

    create_table_query = "CREATE TABLE final_combine ("
    for column_name in header:
        create_table_query += f"`{column_name}` TEXT, "
    create_table_query = create_table_query[:-2]  # Remove the trailing comma and space
    create_table_query += ")"

    cursor = connection.cursor()
    cursor.execute("DROP TABLE IF EXISTS final_combine")
    cursor.execute(create_table_query)

    insert_query = f"INSERT INTO final_combine ({', '.join(['`'+col+'`' for col in header])}) VALUES ({', '.join(['%s'] * len(header))})"

    for row in csv_data:
        cursor.execute(insert_query, row)

    # Commit the changes to the database
    connection.commit()

    cursor.close()

# Close the connection to the MySQL server
connection.close()